In [1]:
# Imports
import pandas as pd

In [2]:
# Definitions
filename_full = r"../../data/data_full.xlsx"
filename = r"../../data/int_agreement (cleaned).xlsx"
sheet_names = ["Arto", "Evanfiya", "James", "Juho", "Sami"]


def load_spreadsheet(filepath,  add_index=False):
    sheets = pd.read_excel(filepath, sheet_name=sheet_names, index_col=0 if add_index else False)
    return sheets

In [3]:
sheets = load_spreadsheet(filename) # Load
df_agreement = pd.concat(sheets.values(), axis=0).dropna(axis=0, subset="title") # Combine sheets
#df_agreement

In [4]:
slice_s = 11
slice_e = 14
eval_cols = df_agreement.columns[slice_s:slice_e]

# Use majority vote to decide label
aggregated = df_agreement[eval_cols].groupby(df_agreement.index).agg(lambda x: x.mode().iloc[0])

df_agreement = df_agreement.drop_duplicates("title") # Make sure only uniques
df_agreement[eval_cols] = aggregated[eval_cols] # Update columns

#df_agreement

In [5]:
sheets = load_spreadsheet(filename_full)
df_full = pd.concat(sheets.values(), axis=0).dropna(axis=0, subset="title")

#Drop first column to match inter-rater cols
df_full = df_full[df_full.columns[1:]]

# df_full

In [6]:
df_complete = pd.concat([df_full, df_agreement], ignore_index=True)
#df_complete.to_csv("complete_dataset.csv", sep=";", index=False)

In [7]:
stats = df_full.columns[slice_s:slice_e]
columns = pd.MultiIndex.from_tuples([
    ("yes", "Count"),
    ("yes", "Percentage"),
    ("partially", "Count"),
    ("partially", "Percentage"),
    ("no", "Count"),
    ("no", "Percentage"),
])

len_df = len(df_complete)
count_yes = df_complete[stats].eq("yes").sum()
count_partially = df_complete[stats].eq("partially").sum()
count_no = df_complete[stats].eq("no").sum()

data = list(zip(
    count_yes, 100 * count_yes / len_df,
    count_partially, 100 * count_partially / len_df,
    count_no, 100 * count_no / len_df
))

df_stats = pd.DataFrame(data, index=stats, columns=columns)
#df_stats # Stats match with the source

print(
    df_stats.to_latex(float_format="%.1f")
    .replace("tabular}", "tabularx}{\\textwidth}", 1)
    .replace("tabular}", "tabularx}")
)


#df[label].where(lambda x : x == "yes").dropna().count() / len(df[label])

\begin{tabularx}{\textwidth}{lrrrrrr}
\toprule
 & \multicolumn{2}{r}{yes} & \multicolumn{2}{r}{partially} & \multicolumn{2}{r}{no} \\
 & Count & Percentage & Count & Percentage & Count & Percentage \\
\midrule
The exercise description matched the selected theme (Yes/Partially/No) & 272 & 96.1 & 7 & 2.5 & 4 & 1.4 \\
The exercise description matched the selected topic (Yes/Partially/No) & 270 & 95.4 & 9 & 3.2 & 4 & 1.4 \\
The exercise description matched the selected concept (Yes/No) & 248 & 87.6 & 0 & 0.0 & 35 & 12.4 \\
\bottomrule
\end{tabularx}



In [8]:
#result2.to_csv("out.csv", sep=";", columns=["title", "topic", "theme", "concept", "problemDescription", "exampleSolution", *eval_cols, label], index=False)
#result2.to_csv("int_agreement_majority_vote.csv", sep=";", columns=["title", "topic", "theme", "concept", "problemDescription", "exampleSolution", *eval_cols, label], index=False)